In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, Dropout, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import Sequence

In [17]:
loaded_model = tf.keras.models.load_model('/kaggle/input/segmentation_unet/keras/default/1/unet_main.keras')

In [5]:
# Custom Data Generator for Segmentation
class SegmentationDataGenerator(Sequence):
    def __init__(self, image_dir, mask_dir, batch_size=32, image_size=(256, 256)):
        self.image_paths = [os.path.join(image_dir, fname) for fname in sorted(os.listdir(image_dir)) if fname.endswith('.jpg')]
        self.mask_paths = [os.path.join(mask_dir, fname) for fname in sorted(os.listdir(mask_dir)) if fname.endswith('.png')]
        self.batch_size = batch_size
        self.image_size = image_size
        self.indexes = np.arange(len(self.image_paths))

        # Check that number of images matches number of masks
        assert len(self.image_paths) == len(self.mask_paths), "Mismatch in number of images and masks"

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_images = [self.load_image(self.image_paths[i]) for i in batch_indexes]
        batch_masks = [self.load_mask(self.mask_paths[i]) for i in batch_indexes]
        
        # Debugging: Print batch shapes
#         print(f"Batch images shape: {np.array(batch_images).shape}")
#         print(f"Batch masks shape: {np.array(batch_masks).shape}")

        return np.array(batch_images), np.array(batch_masks)

    def load_image(self, path):
        img = load_img(path, target_size=self.image_size)
        img_array = img_to_array(img)
        return img_array / 255.0  # Normalize

    def load_mask(self, path):
        mask = load_img(path, target_size=self.image_size, color_mode='grayscale')  # Load mask as grayscale
        mask_array = img_to_array(mask).astype(np.uint8)

        # Convert mask values to integers (assuming 38 is the only non-zero value)
        mask_array = np.where(mask_array == 38, 1, 0)  # Map pixel value 38 to 1, all else to 0

        return mask_array

In [15]:
# Define your validation directories here
val_image_dir = '/kaggle/input/leaf-disease-segmentation-dataset/data/data/images'  # .jpg images
val_mask_dir = '/kaggle/input/leaf-disease-segmentation-dataset/data/data/masks'  # .png masks

# Create validation data generator
val_gen = SegmentationDataGenerator(
    image_dir=val_image_dir,
    mask_dir=val_mask_dir,
    batch_size=16,  # Same batch size as training
    image_size=(256, 256)
)

In [18]:
# Evaluate the model on the validation set
val_loss, val_accuracy = loaded_model.evaluate(
    val_gen,
    steps=len(val_gen)  # Ensure all validation data is evaluated
)

# Print the results
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 136ms/step - accuracy: 0.9572 - loss: 0.1069
Validation Loss: 0.10505835711956024
Validation Accuracy: 0.9581560492515564


In [19]:
from sklearn.metrics import classification_report
import numpy as np

# Function to get predictions and true masks
def get_predictions_and_true_masks(loaded_model, val_gen):
    all_preds = []
    all_true_masks = []

    for i in range(len(val_gen)):
        images, true_masks = val_gen[i]
        preds = loaded_model.predict(images)
        preds = (preds > 0.5).astype(np.uint8)  # Convert probabilities to binary masks

        all_preds.append(preds)
        all_true_masks.append(true_masks)

    # Concatenate all batches
    return np.concatenate(all_preds, axis=0), np.concatenate(all_true_masks, axis=0)

# Get predictions and true masks
predicted_masks, true_masks = get_predictions_and_true_masks(loaded_model, val_gen)

# Flatten the predictions and true masks for classification report
predicted_masks_flat = predicted_masks.flatten()
true_masks_flat = true_masks.flatten()

# Create classification report
report = classification_report(true_masks_flat, predicted_masks_flat, target_names=['Background', 'Disease'])

# Print the classification report
print(report)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━

In [20]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Function to calculate the evaluation metrics
def evaluate_segmentation_metrics(true_masks, predicted_masks):
    # Flatten the arrays
    true_masks_flat = true_masks.flatten()
    predicted_masks_flat = predicted_masks.flatten()
    
    # Calculate TP, TN, FP, FN
    TP = np.sum((true_masks_flat == 1) & (predicted_masks_flat == 1))
    TN = np.sum((true_masks_flat == 0) & (predicted_masks_flat == 0))
    FP = np.sum((true_masks_flat == 0) & (predicted_masks_flat == 1))
    FN = np.sum((true_masks_flat == 1) & (predicted_masks_flat == 0))
    
    # Pixel Accuracy
    total_pixels = true_masks_flat.size
    pixel_accuracy = (TP + TN) / total_pixels
    
    # IoU
    IoU = TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0.0
    
    # Dice Coefficient
    dice_coefficient = (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0.0
    
    # Precision
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    
    # Recall
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    
    # F1 Score
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return {
        'Pixel Accuracy': pixel_accuracy,
        'IoU': IoU,
        'Dice Coefficient': dice_coefficient,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

# Get predictions from the validation set
predicted_masks, true_masks = get_predictions_and_true_masks(loaded_model, val_gen)

# Evaluate the model on the metrics
metrics = evaluate_segmentation_metrics(true_masks, predicted_masks)

# Print the evaluation metrics
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━